In [ ]:
# imports
!pip install pycountry_convert
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.dates as md
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
import numpy as np

In [ ]:
f = open("../input/impact-of-covid19-pandemic-on-the-global-economy/transformed_data.csv", "r")
data = f.readlines()
print(data[0])
print("Totally ", len(data), " records")

data = data[1:]
data = [d.split(",") for d in data]

x = [d[:7] for d in data]
y = [d[8].rstrip("\n") for d in data]
countries = []
for idx in range(len(x)):
    # delete country full name which is redundant
    del x[idx][1]
    # split date to year and month
    date = x[idx][1]
    del x[idx][1]
    ymd = date.split("-")
    countries.append(x[idx][0])
    for num in ymd:
        x[idx].append(num)
    y[idx] = float(y[idx])
print(x[0], y[0])


# one hot encode country code
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(countries)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
for idx in range(len(x)):
    for num in onehot_encoded[idx]:
        x[idx].append(num)
for i in range(len(x)):
    for j in range(len(x[i])):
        try:
            x[i][j] = float(x[i][j])
        except:
            x[i][j] = 0
print(x[0], y[0])

tc = {}
td = {}
for idx in range(len(x)):
    country = countries[idx]
    if country in tc.keys():
        tc[country].append(x[idx][2])
        td[country].append(x[idx][3])
    else:
        tc[country] = [x[idx][2]]
        td[country] = [x[idx][3]]
print(tc['AFG'], td['AFG'])
avg_tc = {}
for country, number in tc.items():
    avg_tc[country] = np.average(number)
avg_td = {}
for country, number in td.items():
    avg_td[country] = np.average(number)
# print(avg_tc)

for idx in range(len(x)):
    x[idx].append(avg_tc[countries[idx]])
    x[idx].append(avg_td[countries[idx]])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np 
test_size = 0.2
# random split
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size) 

# split continously by index, train and test set shouldn't contain the same country
tot = len(x)
x_train = x[int(test_size * tot):]
y_train = y[int(test_size * tot):]
x_test = x[:int(test_size * tot)]
y_test = y[:int(test_size * tot)]


def try_model(model):
    print("Trying", model)
    model.fit(x_train, y_train)
    score = model.score(x_test, y_test)
    result = model.predict(x_test)
    mse = mean_squared_error(y_test, result)
    print(score, mse)
    
    plt.figure()
    plt.plot(np.arange(len(result)), y_test,'go-',label='true value')
    plt.plot(np.arange(len(result)),result,'ro-',label='predict value')
    plt.title('score: %f'%score)
    plt.legend()
    plt.show()

    
from sklearn import tree, linear_model, svm, neighbors, ensemble

# all the models 
models = [
    tree.DecisionTreeRegressor(), 
    linear_model.LinearRegression(), 
#     svm.SVR(), 
    neighbors.KNeighborsRegressor(), 
    ensemble.RandomForestRegressor(n_estimators=500), 
    ensemble.AdaBoostRegressor(n_estimators=500),
    ensemble.GradientBoostingRegressor(n_estimators=500),
    ensemble.BaggingRegressor(),
    tree.ExtraTreeRegressor()
]
# for model in models:
#     try_model(model)

ensemble = [
    linear_model.LinearRegression(),
#     tree.ExtraTreeRegressor(), 
#     ensemble.RandomForestRegressor(n_estimators=20), 
    ensemble.AdaBoostRegressor(n_estimators=500)
]
print("Start train")
trained = [model.fit(x_train, y_train) for model in ensemble]
print("Finish train")
res = [model.predict(x_test) for model in trained]
res = np.array(res)
res = res.mean(axis=0)
mse = mean_squared_error(y_test, res)
print(mse)

In [ ]:
data = pd.read_csv('../input/impact-of-covid19-pandemic-on-the-global-economy/transformed_data.csv')
data.head(5)

In [ ]:
data['DATE'] = pd.to_datetime(data['DATE'], format = '%Y-%m-%d')
data = data.replace([np.inf, -np.inf], np.nan)
data = data.fillna(0)

This dataset contains at least 210 countries, it presented below on the impact of the COVID-19 pandemic on the global economy of each country.

Note :

* CODE
* COUNTRY
* DATE
* HDI : human_development_index
* TC : total_cases
* TD : total_deaths
* STI : stringency_index
* POP : population
* GDPCAP : gdp_per_capita

## Human Development Index (HDI)

The HDI was created to emphasize that people and their capabilities should be the ultimate criteria for assessing the development of a country, not economic growth alone. The HDI can also be used to question national policy choices, asking how two countries with the same level of GNI per capita can end up with different human development outcomes. These contrasts can stimulate debate about government policy priorities. The Human Development Index (HDI) is a summary measure of average achievement in key dimensions of human development: a long and healthy life, being knowledgeable and have a decent standard of living. The HDI is the geometric mean of normalized indices for each of the three dimensions.

The health dimension is assessed by life expectancy at birth, the education dimension is measured by mean of years of schooling for adults aged 25 years and more and expected years of schooling for children of school entering age. The standard of living dimension is measured by gross national income per capita. The HDI uses the logarithm of income, to reflect the diminishing importance of income with increasing GNI. The scores for the three HDI dimension indices are then aggregated into a composite index using geometric mean. Refer to Technical notes for more details.

The HDI simplifies and captures only part of what human development entails. It does not reflect on inequalities, poverty, human security, empowerment, etc. The HDRO offers the other composite indices as broader proxy on some of the key issues of human development, inequality, gender disparity and poverty.

Reference :http://hdr.undp.org/en/content/human-development-index-hdi

In [ ]:
et = data.sort_values(by='HDI', ascending=False)[:7000]
figure = plt.figure(figsize=(15,10))
sns.barplot(y=et.COUNTRY, x=et.HDI)
plt.xticks()
plt.xlabel('HDI')
plt.xlim(xmin=0.8)
plt.ylabel('COUNTRY')
plt.title('Countplot of country by HDI')
plt.show()

### Correlation between HDI and GDPCAP also the HDI mean

In [ ]:
data["HDI"].corr(data["GDPCAP"]), data["HDI"].mean()
data["STI"].corr(data["TC"])

## Stringency Index
- It is among the metrics being used by the Oxford COVID-19 Government Response Tracker.
- The Tracker involves a team of 100 Oxford community members who have continuously updated a database of 17 indicators of government response.
- These indicators examine containment policies such as school and workplace closings, public events, public transport, stay-at-home policies.
- The Stringency Index is a number from 0 to 100 that reflects these indicators. A higher index score indicates a higher level of stringency.
Reference : https://www.civilsdaily.com/news/what-is-stringency-index/

In [ ]:
et = data.sort_values(by='STI', ascending=False)[:2000]
figure = plt.figure(figsize=(15,10))
sns.barplot(y=et.COUNTRY, x=et.STI, palette='rocket')
plt.xticks()
plt.xlim([4.5, 4.65])
plt.xlabel('STI')
plt.ylabel('COUNTRY')
plt.title('Countplot of country by STI')
plt.show()

In [ ]:
data['STI'].mean()

## GDP Per Capita
A country's GDP or gross domestic product is calculated by taking into account the monetary worth of a nation's goods and services after a certain period of time, usually one year. It's a measure of economic activity.

### How Does GDP Per Capita Work?
Essentially, GDP per capita acts as a metric for determining a country's economic output per each person living there. Often times, rich nations with smaller populations tend to have higher per capita GDP. Once you do the math, the wealth is spread among fewer people, which raises a country's GDP.

The fact that the GDP per capita divides a country's economic output by its total population makes it a good measurement of a country's standard of living, especially since it tells you how prosperous a country feels to each of its citizens.

In [ ]:
et = data.sort_values(by='GDPCAP', ascending=False)[:7000]
figure = plt.figure(figsize=(15,10))
sns.barplot(y=et.COUNTRY, x=et.GDPCAP, palette='rocket')
plt.xticks()
plt.xlabel('GDPCAP')
plt.ylabel('COUNTRY')
plt.title('Countplot of country by GDPCAP')
plt.show()

# The impact of the COVID-19 pandemic of each country

In [ ]:
@interact
def data_show(country=sorted(set(data['COUNTRY']))):
    HDI= data[data['COUNTRY']==country]['HDI'].max()*10
    GDPCAP = data[data['COUNTRY']==country]['GDPCAP'].mean()
    POP= data[data['COUNTRY']==country]['POP'].max()

    total1= [HDI, GDPCAP, POP]

    TD= data[data['COUNTRY']==country].groupby(['DATE']).agg({'TD':['sum']})
    TC = data[data['COUNTRY']==country].groupby(['DATE']).agg({'TC':['sum']})
    STI = data[data['COUNTRY']==country].groupby(['DATE']).agg({'STI':['sum']})

    total2 = TD.join(TC).join(STI)

    plt.figure(figsize=(15,10))
    plt.subplot(121)
    names = ['HDI','GDPCAP', 'POP']
    plt.bar(names, total1)
    plt.ylabel("Counts", size=13)

    plt.subplot(122)
    total2.plot(ax=plt.gca(), title=country)
    plt.ylim((0, 16))
    # setting the max value to see the data in the same env
    plt.ylabel("Counts", size=13)

A basic  graph from the total cases vs total deaths also the STI with green

In [ ]:
fig, ax = plt.subplots(figsize = (15, 6))

sns.lineplot(x=data["DATE"], y=data["TC"])
sns.lineplot(x=data["DATE"], y=data["TD"])
sns.lineplot(x=data["DATE"], y=data["STI"])

ax.set_title("Total Cases vs Total Deaths")
ax.set_xlabel("Date")
ax.set_ylabel("Counts")

ax.xaxis.set_major_formatter(md.DateFormatter('%Y-%m-%d'))

The correlation between the total cases and deaths

In [ ]:
data["TC"].corr(data["TD"])

## Total cases by countries over time


In [ ]:
fig, ax = plt.subplots(figsize = (15, 6))

sns.lineplot(data=data, x='DATE', y='TC', hue='COUNTRY', legend=False, palette='rocket')

ax.set_title("Total Cases by countries")
ax.set_xlabel("Date")
ax.set_ylabel("Counts")

ax.xaxis.set_major_formatter(md.DateFormatter('%Y-%m-%d'))

> This is not really helpful so I looked for an other grouping method

Adding ratios between TCs and TDs, after that get the continents as well

In [ ]:
x=[]
for i in range(len(data)):
    if data['TC'].values[i]!=0:
        x.append(data['TD'].values[i]/data['TC'].values[i])
    else:
        x.append(0)
data['Ratio']=x
# adding the ratios between TCs and TDs

In [ ]:
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return cn_continent

In [ ]:
data['CONTINENT'] = data['COUNTRY'].apply(get_continent)

In [ ]:
fig, ax = plt.subplots(figsize = (15, 6))

sns.lineplot(data=data, x='DATE', y='TC', hue='CONTINENT')

ax.set_title("Total Cases by continents")
ax.set_xlabel("Date")
ax.set_ylabel("Counts")

ax.xaxis.set_major_formatter(md.DateFormatter('%Y-%m-%d'))

Much better, but it's still fuzzy. Let's make an interactive version by continent and add the TD and the ration between
* AF - Africa
* AS - Asia
* EU - Europe
* NA - North America
* OC - Oceania
* SA - South America

In [ ]:
@interact
def data_show(continent=sorted(set(data['CONTINENT']))):
    fig, ax = plt.subplots(figsize = (15, 6))

    sns.lineplot(x=data['DATE'], y=data[data['CONTINENT'] == continent]['TC'])
    sns.lineplot(x=data['DATE'], y=data[data['CONTINENT'] == continent]['TD'])
    sns.lineplot(x=data['DATE'], y=data[data['CONTINENT'] == continent]['Ratio'])

    ax.set_title(f"{continent}  TC vs TD and the ratio between them")
    ax.set_xlabel("Date")
    plt.ylim((0, 11))
    # setting the max value to see the data in the same env
    ax.set_ylabel("Counts")